# INTRODUCTION

This notebook implements a game I'll term *Complex Competition* though in reality, the there is nothing truly complex about the topology, (it is not a hypergraph or simplical complex for example).


## Rules

*  gameboard is an undirected graph, connected, graph. Nodes may  be controlled by one or both parties.  One node is marked the goal.
* The defender party starts with control of all nodes except one.
* The attacker party starts with control of one node only.
* Parties take turns. They may:
  * pay A1/D1 cost to observe the control of a node.  
  * Pay A2/D2 cost to establish control of a node. 
  * Pay A3/D3 cost to remove control from a node (only succeeding if they control the node).
  * Pay A4/D4 cost to discovery peers of a node.
  * Pass at no cost.
* They may only act on nodes connected to nodes they control. 
* The attacker party goes first.
* The goal node is assigned a value G.  When the attacker gains control of the goal node, they receive value G and the defender loses value G.
* The game is over when both parties stop playing.  Once a  party has stopped playing,t hey may not start again.


## Potential Player Strategies

* Attacker random: Choose a peer at random and control it
* Attacker random no-backtrack:  Choose a random node not previously controlled and control it.
* Attacker shortest-path: (With full knowledge of the topology and goal location) calculate the shortest path to the goal and control it in order.
* Attacker DFS: Use a depth first search to identify nodes to control
* Attacker inspector: Attacker checks to ensure they have control before taking action
* Attacker low degree:  Attacker chooses a peer to control based on the degree of the node.
* Defender do-nothing: Defender stops immediately
* Defender check and mitigate: Defender picks a random node to check and mitigate.
* Defender centrality: Defender picks nodes based on centrality to mitigate
* Defender shortest path: Defender picks nodes on the shortest path to check and mitigate
* Defender peer: Defender finds a compromised node and then inspects peers to find and mitigate attacker controlled nodes

## Options

* Different topologies
* Different goal locations
* Different goal values
* Different costs
* Different player strategies
* Different actor starting nodes
* Both parties maybe limited in their awareness of the game board by setting 'visibility' in the game.  'visibility' of zero equates to players only being aware of the topology they control at the beginning of the game.
* Additional parameters may be passed to the player classes as appropriate through the game initialization
* Multiple goals

## Research Questions

* When should the defender stop defending / how much should they spend on defense?
* How does the location of the attacker affect their cost to realize the goal?
* How does the goal location affect the attacker's cost to realize the goal?
* How do different topologies affect the attacker and defender costs?
* How do different costs affect the attacker's chance of reaching the goal?
* How do different costs affect the attacker's chance of reaching the goal?
* What is the relationship between topology, attacker strategy, attacker action cost, and goal value?

## Potential Improvements

* Add more information to the nodes to help players choose actions
* Probability of success per edge
* Cost of action per node
* Replace the undirected graph with a directed graph
* Different value for the attacker and defender for achieving the goal.
* Separating the impact cost to the defender from the goal and having them on separate nodes
* Allow the defender to take more than one action per round
* set per edge success probabilities and costs
* Create actin probabilities
* Allow the defender to pay to increase attacker action cost (potentially per edge).
* Allow the defender to pay to decrease the action success probability (potentially per edge).
* Model monitoring

## Changelog

* 0.0.9004
  * changed goal value to be stored on the topology so that different nodes can have different values.
  * goal() function now received both the node and value
  * defender goal() function is not called unless the node is inspected.
* 0.0.9003
  * Experimental MESA design (separate branch)
* 0.0.9002
  * Updated topology for users to connect through mail and proxy to internet rather than directly (Primarily to give defense better monitoring options
* 0.0.9001
  * Added documentation
  * Added ability to pass initialization parameters to player classes
* 0.0.9000
  * Moved stoping to player determined.
  * Added visibility parameter
* Initial 
  * Basic functionality

# SETUP

In [549]:
import networkx as nx
import itertools
import copy
import random
# visualizing
import altair as alt
import pandas as pd

# GAME MODEL

In [595]:
class complex_competition:
    """
    A class used to represent a single instance of a game

    ...

    Attributes
    ----------
    topology : nx.classes.graph.Graph
        The network topology on which the game is played
    attacker/defender : player classes
        Must implement initialization with at least a topology
        Must implement a turn() function taking nothing and returning a tuple of strings of (node, action). Action may be 'inspect', 'control', 'boot', 'pass', or 'stop'.
        Must implement an inspect(node, properties) function taking a node name string and dictionary of properties
        Must implement a discover(node, edges) function taking a list of edge tuples.
        Must implement a goal(node) function taking a node as a string representing a goal reached by the attacker.
    costs : dictionary
        Costs with keys for attacker/defender_inspect/control/boot/discover costs as ints.
    turn: str
        The name of the current player (attacker or defender)
    value: dict
        A dictionary of {"attacker": int, "defender": int} representing the current value of each player
    actions: list()
        A list of dictionaries representing the log of actions in the game
    goals_achieved: dict
        A dictionary with key of node and value of goal value. Used to capture goals achived
    stop: dict
        A dict of {"attacker": bool, "defender": bool} represnting which players have stopped
    max_iter: int
        The maximum number of iterations to play if either side does not stop
    props_to_share: list
        A list of node properties to share upon inspection

    Methods
    -------
    play()
        Starts game and runs it until max_iter or both players stop.
    """
    
    
    topology = None
    attacker = None
    defender = None
    costs = None
    turn = None
    value = None
    goals_achieved = None
    actions = None
    stop = None
    max_iter = None
    props_to_share = ["attacker_control", "defender_control"]

    def __init__(self, a, d, 
                 topology = None, 
                 goal_value=10, 
                 costs=None, 
                 visibility=None, 
                 max_iter=10000, 
                 a_params = {},
                 d_params = {}):
        """
        Parameters
        ----------
        topology : None, str, or nx.classes.graph.Graph
            Topology to use for the game
            If None, the default topology will be used
            If str, the str will be opened as a graphml file
            If nx.classes.graph.Graph, will be loaded directly
        goal_value : int
            the value associated with winning a goal
        costs : dictionary
            Costs with keys for attacker/defender_inspect/control/boot/discover costs as ints.
        visibility: None or int
            If None, visibility is infinite.
            If a positive integer, players will only be given the subset of the topology they control initially
        max_iter: int
            The maximum number of iterations to play if either side does not stop
        a_params: dict
            A dictionary of additional params to pass to the attacker class when initializing it
        d_params: dict
            A dictionary of additional params to pass to the defender class when initializing it
        """
        
        self.value = {"attacker": 0, "defender": 0}
        self.turn = "attacker"
        self.stop = {'attacker': False, 'defender': False}
        if costs is None:
            self.costs = {
                "attacker_inspect": 1,
                "defender_inspect": 1,
                "attacker_control": 1,
                "defender_control": 1,
                "attacker_boot": 1,
                "defender_boot": 1,
                "attacker_discover": 1,
                "defender_discover": 1
            }
        self.max_iter = max_iter # max iterations
        self.actions = list()
        self.goals_achieved = dict()
        
        ### Get a topology
        if topology is None:
            g = nx.Graph()
            
            ## outside node
            outside = ["internet"]
            # DMZ: 2 webapp and 2 database
            dmz = ['webapp1', 'db1', "webapp2", "db2", "mail", "proxy"]
            # Userlan: 5 users
            users = ["user1", "user2", "user3", "user4", "user5"]
            # 3 mgmt systems
            mgmt = ["mgmt1", "mgmt2", "mgmt3"]
            # 3 infrastructure systems
            infra = ["infra1", "infra2", "infra3"]
            # Servers (7 servers in three permission groups)
            svrs1 = ["svr1_1", "svr1_2", "svr1_3"] 
            svrs2 = ["svr2_1", "svr2_2", "svr2_3"]
            svrs3 = ["svr3_1"]
            # PCI: 3 PCI related servers
            pci = ["pci1", "pci2", "pci3"]
            # add nodes
            g.add_nodes_from(dmz + users + mgmt + infra + svrs1 + svrs2 + svrs3 + pci)
            
            ## add edges
            # internet access to dmz webapps
            g.add_edges_from([("internet", "webapp1"), ("internet", "webapp2"), 
                              ("internet", "mail"), ("internet", "proxy")])
            # internet access to all clients
            nx.add_star(g, ["mail"] + users)
            nx.add_star(g, ["proxy"] + users)
            # Webapps are connected to their database
            g.add_edges_from([('webapp1', 'db1'), ("webapp2", "db2")])
            # all clients are in the same security domain
            edges = itertools.combinations(users, 2)
            g.add_edges_from(edges)
            # add server groups
            edges = itertools.combinations(svrs1, 2)
            g.add_edges_from(edges)
            edges = itertools.combinations(svrs2, 2)
            g.add_edges_from(edges)
            # Connect users to svr 1 in each group
            nx.add_star(g, svrs1[:1] + users)
            nx.add_star(g, svrs2[:1] + users)
            nx.add_star(g, svrs3[:1] + users)
            # Connect users to webapps
            nx.add_star(g, ["webapp1"] + users)
            nx.add_star(g, ["webapp2"] + users)
            # mgmt is only accessable from 1 user but accesses every server
            nx.add_star(g, users[-1:] + mgmt)
            for node in mgmt:
                nx.add_star(g, [node] + dmz + svrs1 + svrs2 + svrs3 + infra)
            # infra is only accessable from mgmt, but connects to all other servers
            for node in infra:
                nx.add_star(g, [node] + dmz + svrs1 + svrs2 + svrs3)
            # PCI connects to 1 server and then to itself
            g.add_edge(svrs3[0], pci[0])
            edges = itertools.combinations(pci, 2)
            g.add_edges_from(edges)
            g.add_edge("webapp2", svrs3[0]) # webapp connected to PCI
            ## Add goal to PCI environment
            attr = {node:(goal_value if node == "pci3" else 0) for node in g.nodes()}
            nx.set_node_attributes(g, attr, 'goal')
            ## Set initial control: controller controls everything but internet, attacker also controls the internet
            attr = {node:(True if node == "internet" else False) for node in g.nodes()}
            nx.set_node_attributes(g, attr, 'attacker_control')
            attr = {node:(True if node != "internet" else False) for node in g.nodes()}
            nx.set_node_attributes(g, attr, 'defender_control')
        elif type(topology) == str:
            g = nx.read_graphml(topology)
        elif type(topology) == nx.classes.graph.Graph:
            g = topology
        else:
            raise TypeError("Topology is not None, a filename, or an undirected Graph.")
        self.topology = g

        ### Register actor classes and initialize
        g_atk = copy.deepcopy(g)
        nx.set_node_attributes(g_atk, None, "defender_control")
        g_atk = self.remove_invisible(g_atk, visibility, "attacker")
        self.attacker = a(topology = g_atk, **a_params)
        g_def = copy.deepcopy(g)
        nx.set_node_attributes(g_def, None, "attacker_control")
        g_def = self.remove_invisible(g_def, visibility, "defender")
        self.defender = d(topology = g_def, **d_params)

        
        
    def remove_invisible(self, g, visibility, player):
        # restrict visibility
        if type(visibility) == int and visibility >= 0:
            ctrl = nx.get_node_attributes(self.topology, "{0}_control".format(player))
            ctrl = [k for k,v in ctrl.items() if not v]
            # only aware of existing nodes
            if visibility == 0:
                g.remove_nodes_from(set(g.nodes()).difference(ctrl))
            elif visibility == 1:
                peers = set(ctrl)
                for node in ctrl:
                    peers = peers.union([n for n in self.topology.neighbors(node)])
                g.remove_nodes_from(set(g.nodes()).difference(peers))
            else:
                # if we're using arbitrary lengths, this gets harder
                # get all the paths
                p = dict(nx.shortest_path_length(g))
                p = {k:v for k,v in p if k in ctrl}
                lengths = dict()
                # get the lengths from controlled nodes
                for k,v in p.items():
                    for k2,v2 in v.items():
                        if k2 not in lengths:
                            lengths[k2] = set()
                        lengths[k2].add(v2)
                # subset to only those within visibility
                visible = [node for node,value in lengths if min(value) <= visibility]
                # remove the invisible nodes
                g.remove_nodes_from(set(g.nodes()).difference(ctrl + visible))
        return g

    def play(self):
        """Switches between players, calling their turn() function and implementing the result.


        Parameters
        ----------
        None

        Raises
        ------
        NotImplementedError
            If no sound is set for the animal or passed in as a
            parameter.
        """
        play = True
        iterations = 0
        
        while play and iterations < self.max_iter:
            iterations += 1
            
            # get player
            if self.turn == "attacker":
                player = self.attacker
            else:
                player = self.defender


            # get action
            action = player.turn()
            
            # if pass, skip the rest
            if action[1] == "pass":
                self.actions.append({
                    "player": self.turn,
                    "node": action[0],
                    "action": action[1],
                    "result": {"value": 0}

                })
                continue
            elif action[1] == "stop":
                self.actions.append({
                    "player": self.turn,
                    "node": action[0],
                    "action": action[1],
                    "result": {"value": 0}

                })
                self.stop[self.turn] = True
                # Is game over?
                if self.stop['attacker'] and self.stop['defender']:
                    play = False
                    break
                else:
                    self.turn = {"attacker": "defender", "defender": "attacker"}[self.turn] # return control to other player
                    continue

            ## Adjudicate action
            # get currect actor's adjacent nodes
            ctrl = nx.get_node_attributes(self.topology, "{0}_control".format(self.turn))
            ctrl = [k for k,v in ctrl.items() if v]
            peers = set(ctrl)
            for node in ctrl:
                peers = peers.union([n for n in self.topology.neighbors(node)])
            if action[0] not in peers and action[1] != "pass":
                print("Actor {0} attempted to act on node {1} but it is not a peer so action had no effect.".format(self.turn, action[0]))
                if action[1] == "inspect":
                    result = {"attacker_control": None, "defender_control": None, 
                              "value": -self.costs["{0}_inspect".format(self.turn)]}
                    self.value[self.turn] += result["value"]
                elif action[1] == "control":
                    result = {"flip": None, "value": -self.costs["{0}_control".format(self.turn)]}
                    self.value[self.turn] += result["value"]
                elif action[1] == "boot":
                    result = {"flip": None, "value": -self.costs["{0}_boot".format(self.turn)]}
                    self.value[self.turn] += result['value']
                elif action[1] == "discover":
                    result = {"discover": None, "value": -self.costs["{0}_discover".format(self.turn)]}
                    self.value[self.turn] += result['value']
                
            elif action[1] == "boot" and not self.topology.nodes[action[0]]["{0}_control".format(self.turn)]:
                print("Actor {0} attempted to boot from node {1} which they do not control, having no effect.".format(self.turn, action[0]))
                result = {"flip": None, "value": -self.costs["{0}_boot".format(self.turn)]}
                self.value[self.turn] += result['value']

            # enact action
            elif action[1] == "inspect":
                # capture result of action
                node_properties = self.topology.nodes[action[0]]
                node_properties = {k:v for k,v in node_properties.items() if k in self.props_to_share}
                result = {
                    "properties": node_properties,
                    "value": -self.costs["{0}_inspect".format(self.turn)]
                }
                #incriment value
                self.value[self.turn] += result['value']
                # inform player
                player.inspect(
                    node=action[0],
                    properties = node_properties
                )
                if action[0] in self.goals_achieved:
                    player.goal(action[0], self.goals_achieved[action[0]])
            elif action[1] == "control":
                # capture result of action
                result = {
                    "flip": not self.topology.nodes[action[0]]["{0}_control".format(self.turn)],
                    "value": -self.costs["{0}_control".format(self.turn)]
                }
                # incriment value
                self.value[self.turn] += result["value"]
                # update game state
                self.topology.nodes[action[0]]["{0}_control".format(self.turn)] = True
                
                # Adjudicate control of a goal
                if self.topology.nodes[action[0]]['goal'] > 0 and self.turn == "attacker":
                    goal_value = self.topology.nodes[action[0]]['goal']
                    # record the goal achievement
                    # to get log in order, we record the initial action early ...
                    self.actions.append({
                        "player": self.turn,
                        "node": action[0],
                        "action": action[1],
                        "result": result
                    })
                    # ... record an action for the defender's loss ...
                    self.actions.append({
                        "player": "defender", 
                        "node": action[0], 
                        "action": "goal", 
                        "result": {"value": -goal_value}})
                    # ... and set the values so the  log line below captures attacker's gain ....
                    result = {"value": goal_value}
                    action = (action[0], "goal")
                    # Update values
                    self.value['attacker'] += goal_value
                    self.value['defender'] -= goal_value
                    # Let attacker know a goal has been achieved
                    self.attacker.goal(action[0], goal_value)
                    # store so defender can be informed
                    self.goals_achieved[action[0]] = goal_value
                    # Remove goal as it has been achieved
                    self.topology.nodes[action[0]]['goal'] = 0
            elif action[1] == "boot":
                # capture result of action
                result = {
                    "flip": self.topology.nodes[action[0]]["{0}_control".format(self.turn)],
                    "value": -self.costs["{0}_boot".format(self.turn)]
                }
                # incriment value
                self.value[self.turn] += result['value']
                # update game state
                self.topology.nodes[action[0]]["{0}_control".format(
                    {"attacker": "defender", "defender": "attacker"}[self.turn])] = False
            elif action[1] == "discover":
                # capture result of action
                result = {
                    "edges": list(g.edges(action[0])),
                    "value": -self.costs["{0}_discover".format(self.turn)]
                }
                #incriment value
                self.value[self.turn] += result['value']
                # inform player
                player.discover(action[0], g.edges(action[0]))
            else: # pass
                pass

            
            # keep a log of what happened this game
            self.actions.append({
                "player": self.turn,
                "node": action[0],
                "action": action[1],
                "result": result

            })
            
#            for node,data in self.topology.nodes(data=True):
#                # attacker wins
#                if data['goal'] and data['attacker_control']:
#                    play = False
#                    self.value['attacker'] += self.goal_value
#                    self.value['defender'] -= self.goal_value
#                    self.actions.append({"player": "attacker", "action":"done", "result": {"value": self.game_value}})
#                    self.actions.append({"player": "defender", "action":"done", "result": {"value": -self.game_value}})
#                    break
#            # attacker goes bust, defender wins
#            if self.stop_at_loss:
#                if play and self.value['attacker'] <= -self.goal_value:
#                    play = False
#                    self.actions.append({"player": "attacker", "action":"done", "result": {"value": 0}})
#                    self.actions.append({"player": "defender", "action":"done", "result": {"value": 0}})

            if play:
                # update the turn if the other side is still playing
                if self.turn != "attacker" and self.turn != "defender":
                    raise ValueError("Turn player {0} is not a valid player.".format(self.turn))
                if self.turn == "attacker" and not self.stop["defender"]:
                    self.turn = "defender"
                elif self.turn == "defender" and not self.stop["attacker"]:
                    self.turn = "attacker"
                    

# PLAYERS

## Attackers

### Random Search  
Attacker chooses a known peer at random and attacks it

In [585]:
### Define random search Attacker
class attacker_c:
    """
    A class used to represent a player
    
    This class implements an attacker randomly controlling peer nodes until the goal is reached.

    ...

    Attributes
    ----------
    topology : nx.classes.graph.Graph
        The network topology on which the game is played as observed by the player
    value : int
        Player's internal representation of the cost of the game
    cost : dict
        Dictionary like {'inspect':1, 'control':1, 'boot':1, 'discover': 1} representing player's understanding of costs of each action
    goal_achieved : bool
        Has the game reported a goal being acheived by the attacker.

    Methods
    -------
    turn()
        chooses an action and returns tuple (node, action)
    inspect(node, properties):
        updates the internal understanding of the topology with the inspected properties on the associated node.
    discover(node, edges):
        updates the internal understanding of the topology with the discovered edges for the associated node.
    goal(node):
        updates the goal as achieved
    """
    topology = None
    goal_achieved = None
    value = None
    cost = None

    def __init__(self, topology, value=100, cost=None):
        """
        Parameters
        ----------
        topology : nx.classes.graph.Graph
            The network topology on which the game is played as observed by the player
        value : int
            Player's initial starting value
        cost : dict
            Dictionary like {'inspect':1, 'control':1, 'boot':1, 'discover': 1} representing player's understanding of costs of each action
        """
        self.topology = topology
        self.goal_achieved = False
        self.value = value
        if cost is None:
            self.cost = {'inspect':1, 'control':1, 'boot':1, 'discover': 1}
        else:
            self.cost = cost

    def turn(self):
        """Implements the player's strategy.

        Parameters
        ----------
        None

        Raises
        ------
        TBD
        """
        if self.goal_achieved:
            return (None, "stop")
        
        if self.value <= 0:
            return (None, "stop")
        
        # get nodes in play
        ctrl = nx.get_node_attributes(self.topology, "attacker_control")
        ctrl = [k for k,v in ctrl.items() if v]
        peers = set()
        for node in ctrl:
            peers = peers.union([n for n in self.topology.neighbors(node)])
        # choose a random target to take over
        potential_targets = list(peers.difference(ctrl))
        if len(potential_targets) <= 0:
            return (None, "stop")
        else:
            target = random.choice(potential_targets)
            self.value -= self.cost['control']
            return (target, "control")

    def inspect(self, node, properties):
        """Updates the player's understanding of the topology with the additional node properties.

        Parameters
        ----------
        node : str
            The inspected node name
        properties: dict
            The properties of the inspected node (note: the game may limit shared properties)

        Raises
        ------
        TBD
        """
        self.topology.nodes[node]['attacker_control'] = properties.get('attacker_control', None)
        self.topology.nodes[node]['defender_control'] = properties.get('defender_control', None)
        
    def discover(self, node, edges):
        """Updates the player's understanding of the topology with the additional edges.

        Parameters
        ----------
        node : str
            The discovered node name
        edges: list
            The list of tuples of edges associated with the discovered node

        Raises
        ------
        TBD
        """
        self.topology.add_edges_from(edges)
        
    def goal(self, node, value):
        """Updates the player's understanding of the topology with the attacker control of the associated node.

        Parameters
        ----------
        node : str
            Identifies that the attacker has controlled the associated goal node.
        value: float
            The value of the goal

        Raises
        ------
        TBD
        """
        self.goal_achieved = True

### Random no backtrack  
Attacker chooses a random peer not previously visited

In [586]:
### Define random search Attacker
class attacker_no_backtrack_c:
    """
    A class used to represent a player
    
    This class implements an attacker randomly controlling peer nodes not controlled until the goal is reached.

    ...

    Attributes
    ----------
    topology : nx.classes.graph.Graph
        The network topology on which the game is played as observed by the player
    value : int
        Player's internal representation of the cost of the game
    cost : dict
        Dictionary like {'inspect':1, 'control':1, 'boot':1, 'discover': 1} representing player's understanding of costs of each action
    goal_achieved : bool
        Has the game reported a goal being acheived by the attacker.

    Methods
    -------
    turn()
        chooses an action and returns tuple (node, action)
    inspect(node, properties):
        updates the internal understanding of the topology with the inspected properties on the associated node.
    discover(node, edges):
        updates the internal understanding of the topology with the discovered edges for the associated node.
    goal(node):
        updates the goal as achieved
    """
    
    topology = None
    goal_achieved = None
    value = None
    cost = None

    def __init__(self, topology, value=100, cost=None):
        """
        Parameters
        ----------
        topology : nx.classes.graph.Graph
            The network topology on which the game is played as observed by the player
        value : int
            Player's initial starting value
        cost : dict
            Dictionary like {'inspect':1, 'control':1, 'boot':1, 'discover': 1} representing player's understanding of costs of each action
        """
        self.topology = topology
        self.goal_achieved = False
        self.value = value
        if cost is None:
            self.cost = {'inspect':1, 'control':1, 'boot':1, 'discover': 1}
        else:
            self.cost = cost

    def turn(self):
        """Implements the player's strategy.

        Parameters
        ----------
        None

        Raises
        ------
        TBD
        """
        if self.goal_achieved:
            return (None, "stop")
        
        if self.value <= 0:
            return (None, "stop")
        
        # get nodes in play
        ctrl = nx.get_node_attributes(self.topology, "attacker_control")
        ctrl = [k for k,v in ctrl.items() if v]
        peers = set()
        for node in ctrl:
            peers = peers.union([n for n in self.topology.neighbors(node)])
        # choose a random target to take over
        potential_targets = list(peers.difference(ctrl))
        if len(potential_targets) <= 0:
            return (None, "stop")
        else:
            target = random.choice(potential_targets)
            self.topology.nodes[target]["attacker_control"] = True
            self.value -= self.cost['control']
            return (target, "control")

    def inspect(self, node, properties):
        """Updates the player's understanding of the topology with the additional node properties.

        Parameters
        ----------
        node : str
            The inspected node name
        properties: dict
            The properties of the inspected node (note: the game may limit shared properties)

        Raises
        ------
        TBD
        """
        self.topology.nodes[node]['attacker_control'] = properties.get('attacker_control', None)
        self.topology.nodes[node]['defender_control'] = properties.get('defender_control', None)
        
    def discover(self, node, edges):
        """Updates the player's understanding of the topology with the additional edges.

        Parameters
        ----------
        node : str
            The discovered node name
        edges: list
            The list of tuples of edges associated with the discovered node

        Raises
        ------
        TBD
        """
        self.topology.add_edges_from(edges)
        
    def goal(self, node, value):
        """Updates the player's understanding of the topology with the attacker control of the associated node.

        Parameters
        ----------
        node : str
            Identifies that the attacker has controlled the associated goal node.
        value: float
            The value of the goal

        Raises
        ------
        TBD
        """
        self.goal_achieved = True

### Shortest Path  
With apriori knowledge of the goal, attacker goes directly to it.

In [587]:
### Define random search Attacker
class attacker_shortest_path_c:
    """
    A class used to represent a player
    
    This class implements an attacker that calculates the shortest path to the goal and takes it.
    This requires the attacker to know the entire topology and goal location.

    ...

    Attributes
    ----------
    topology : nx.classes.graph.Graph
        The network topology on which the game is played as observed by the player
    value : int
        Player's internal representation of the cost of the game
    cost : dict
        Dictionary like {'inspect':1, 'control':1, 'boot':1, 'discover': 1} representing player's understanding of costs of each action
    goal_achieved : bool
        Has the game reported a goal being acheived by the attacker.

    Methods
    -------
    turn()
        chooses an action and returns tuple (node, action)
    inspect(node, properties):
        updates the internal understanding of the topology with the inspected properties on the associated node.
    discover(node, edges):
        updates the internal understanding of the topology with the discovered edges for the associated node.
    goal(node):
        updates the goal as achieved
    """
    
    topology = None
    steps = None
    goal_achieved = None

    def __init__(self, topology, value=None, cost=None):
        """
        Parameters
        ----------
        topology : nx.classes.graph.Graph
            The network topology on which the game is played as observed by the player
        value : int
            Player's initial starting value
        cost : dict
            Dictionary like {'inspect':1, 'control':1, 'boot':1, 'discover': 1} representing player's understanding of costs of each action
        """
        self.topology = topology
        self.steps = nx.shortest_path(game.topology, "internet", "pci3") # cheat
        self.steps.reverse()
        self.goal_achieved = False

    def turn(self):
        """Implements the player's strategy.

        Parameters
        ----------
        None

        Raises
        ------
        TBD
        """
        if self.goal_achieved:
            return (None, "stop")
        
        try:
            target = self.steps.pop()
            return (target, "control")
        except IndexError:
            return (None, "stop")

    def inspect(self, node, properties):
        """Updates the player's understanding of the topology with the additional node properties.

        Parameters
        ----------
        node : str
            The inspected node name
        properties: dict
            The properties of the inspected node (note: the game may limit shared properties)

        Raises
        ------
        TBD
        """
        self.topology.nodes[node]['attacker_control'] = properties.get('attacker_control', None)
        self.topology.nodes[node]['defender_control'] = properties.get('defender_control', None)
        
    def discover(self, node, edges):
        """Updates the player's understanding of the topology with the additional edges.

        Parameters
        ----------
        node : str
            The discovered node name
        edges: list
            The list of tuples of edges associated with the discovered node

        Raises
        ------
        TBD
        """
        self.topology.add_edges_from(edges)
        
    def goal(self, node, value):
        """Updates the player's understanding of the topology with the attacker control of the associated node.

        Parameters
        ----------
        node : str
            Identifies that the attacker has controlled the associated goal node.
        value: float
            The value of the goal

        Raises
        ------
        TBD
        """
        self.goal_achieved = True

### DFS Attack  
Attacker search using a depth-first-search

In [588]:
class attacker_dfs_c:
    """
    A class used to represent a player
    
    This class implements a depth first seach for the goal

    ...

    Attributes
    ----------
    topology : nx.classes.graph.Graph
        The network topology on which the game is played as observed by the player
    value : int
        Player's internal representation of the cost of the game
    cost : dict
        Dictionary like {'inspect':1, 'control':1, 'boot':1, 'discover': 1} representing player's understanding of costs of each action
    goal_achieved : bool
        Has the game reported a goal being acheived by the attacker.

    Methods
    -------
    turn()
        chooses an action and returns tuple (node, action)
    inspect(node, properties):
        updates the internal understanding of the topology with the inspected properties on the associated node.
    discover(node, edges):
        updates the internal understanding of the topology with the discovered edges for the associated node.
    goal(node):
        updates the goal as achieved
    """
    
    topology = None
    goal_achieved = None
    value = None
    cost = None
    to_explore = None

    def __init__(self, topology, value=0, cost=None):
        """
        Parameters
        ----------
        topology : nx.classes.graph.Graph
            The network topology on which the game is played as observed by the player
        value : int
            Player's initial starting value
        cost : dict
            Dictionary like {'inspect':1, 'control':1, 'boot':1, 'discover': 1} representing player's understanding of costs of each action
        """
        #raise NotImplementedError("DFS attacker not yet implemented")
        
        self.topology = topology
        self.goal_achieved = False
        self.value = value
        self.to_explore = list()
        if cost is None:
            self.cost = {'inspect':1, 'control':1, 'boot':1, 'discover': 1}
        else:
            self.cost = cost

    def turn(self):
        """Implements the player's strategy.

        Parameters
        ----------
        None

        Raises
        ------
        TBD
        """
        if self.goal_achieved:
            return (None, "stop")
        
        # get nodes in play
        ctrl = nx.get_node_attributes(self.topology, "attacker_control")
        ctrl = [k for k,v in ctrl.items() if v]
        for node in ctrl:
            peers = list(self.topology.neighbors(node))
            random.shuffle(peers)
            for peer in peers:
                if peer not in ctrl and peer not in self.to_explore:
                    self.to_explore.append(peer)
        # choose a random target to take over
        if len(self.to_explore) <= 0:
            return (None, "stop")
        target = self.to_explore.pop()
        self.topology.nodes[target]["attacker_control"] = True
        self.value -= self.cost['control']
        return (target, "control")

    def inspect(self, node, properties):
        """Updates the player's understanding of the topology with the additional node properties.

        Parameters
        ----------
        node : str
            The inspected node name
        properties: dict
            The properties of the inspected node (note: the game may limit shared properties)

        Raises
        ------
        TBD
        """
        self.topology.nodes[node]['attacker_control'] = properties.get('attacker_control', None)
        self.topology.nodes[node]['defender_control'] = properties.get('defender_control', None)
        
    def discover(self, node, edges):
        """Updates the player's understanding of the topology with the additional edges.

        Parameters
        ----------
        node : str
            The discovered node name
        edges: list
            The list of tuples of edges associated with the discovered node

        Raises
        ------
        TBD
        """
        self.topology.add_edges_from(edges)
        
    def goal(self, node, value):
        """Updates the player's understanding of the topology with the attacker control of the associated node.

        Parameters
        ----------
        node : str
            Identifies that the attacker has controlled the associated goal node.
        value: float
            The value of the goal

        Raises
        ------
        TBD
        """
        self.goal_achieved = True

### Least-knowledge attack  
Attacker looks for nodes with the minimum number of connections to controlled nodes and searches them.  (The concept being that the fewer interconnections, the more likely the node is to be a bridge to an isolated part of the network.)

In [589]:
### Define random search Attacker
class attacker_least_knowledge_c:
    """
    A class used to represent a player
    
    This class implements an attacker randomly controlling peer nodes not controlled until the goal is reached.

    ...

    Attributes
    ----------
    topology : nx.classes.graph.Graph
        The network topology on which the game is played as observed by the player
    value : int
        Player's internal representation of the cost of the game
    cost : dict
        Dictionary like {'inspect':1, 'control':1, 'boot':1, 'discover': 1} representing player's understanding of costs of each action
    goal_achieved : bool
        Has the game reported a goal being acheived by the attacker.

    Methods
    -------
    turn()
        chooses an action and returns tuple (node, action)
    inspect(node, properties):
        updates the internal understanding of the topology with the inspected properties on the associated node.
    discover(node, edges):
        updates the internal understanding of the topology with the discovered edges for the associated node.
    goal(node):
        updates the goal as achieved
    """
    
    topology = None
    goal_achieved = None
    value = None
    cost = None

    def __init__(self, topology, value=100, cost=None):
        """
        Parameters
        ----------
        topology : nx.classes.graph.Graph
            The network topology on which the game is played as observed by the player
        value : int
            Player's initial starting value
        cost : dict
            Dictionary like {'inspect':1, 'control':1, 'boot':1, 'discover': 1} representing player's understanding of costs of each action
        """
        self.topology = topology
        self.goal_achieved = False
        self.value = value
        if cost is None:
            self.cost = {'inspect':1, 'control':1, 'boot':1, 'discover': 1}
        else:
            self.cost = cost

    def turn(self):
        """Implements the player's strategy.

        Parameters
        ----------
        None

        Raises
        ------
        TBD
        """
        if self.goal_achieved:
            return (None, "stop")
        
        # get nodes in play
        ctrl = nx.get_node_attributes(self.topology, "attacker_control")
        ctrl = [k for k,v in ctrl.items() if v]
        potential_targets = dict()
        for node in ctrl:
            peers = [n for n in self.topology.neighbors(node)]
            for peer in peers:
                if peer not in ctrl:
                    if peer not in potential_targets:
                        potential_targets[peer] = 1
                    else:
                        potential_targets[peer] += 1
        if len(potential_targets) <= 0:
            return (None, "stop")
        else:
            # randomly choose from the peers with the minimum of controlled peers
            target = random.choice([k for k,v in potential_targets.items() if v == min(potential_targets.values())])
            self.topology.nodes[target]["attacker_control"] = True
            self.value -= self.cost['control']
            return (target, "control")

    def inspect(self, node, properties):
        """Updates the player's understanding of the topology with the additional node properties.

        Parameters
        ----------
        node : str
            The inspected node name
        properties: dict
            The properties of the inspected node (note: the game may limit shared properties)

        Raises
        ------
        TBD
        """
        self.topology.nodes[node]['attacker_control'] = properties.get('attacker_control', None)
        self.topology.nodes[node]['defender_control'] = properties.get('defender_control', None)
        
    def discover(self, node, edges):
        """Updates the player's understanding of the topology with the additional edges.

        Parameters
        ----------
        node : str
            The discovered node name
        edges: list
            The list of tuples of edges associated with the discovered node

        Raises
        ------
        TBD
        """
        self.topology.add_edges_from(edges)
        
    def goal(self, node, value):
        """Updates the player's understanding of the topology with the attacker control of the associated node.

        Parameters
        ----------
        node : str
            Identifies that the attacker has controlled the associated goal node.
        value: float
            The value of the goal

        Raises
        ------
        TBD
        """
        self.goal_achieved = True

### Short attacks  
Attacker takes 3 actions and quits

In [590]:
### Define random search Attacker
class attacker_three_and_out_c:
    """
    A class used to represent a player
    
    This class implements an attacker randomly controlling peer nodes not controlled until a three actions or the goal is reached. (Whichever comes first.)

    ...

    Attributes
    ----------
    topology : nx.classes.graph.Graph
        The network topology on which the game is played as observed by the player
    value : int
        Player's internal representation of the cost of the game
    cost : dict
        Dictionary like {'inspect':1, 'control':1, 'boot':1, 'discover': 1} representing player's understanding of costs of each action
    goal_achieved : bool
        Has the game reported a goal being acheived by the attacker.

    Methods
    -------
    turn()
        chooses an action and returns tuple (node, action)
    inspect(node, properties):
        updates the internal understanding of the topology with the inspected properties on the associated node.
    discover(node, edges):
        updates the internal understanding of the topology with the discovered edges for the associated node.
    goal(node):
        updates the goal as achieved
    """
    
    topology = None
    goal_achieved = None
    value = None
    cost = None
    steps = None

    def __init__(self, topology, value=0, cost=None):
        """
        Parameters
        ----------
        topology : nx.classes.graph.Graph
            The network topology on which the game is played as observed by the player
        value : int
            Player's initial starting value
        cost : dict
            Dictionary like {'inspect':1, 'control':1, 'boot':1, 'discover': 1} representing player's understanding of costs of each action
        """
        self.topology = topology
        self.goal_achieved = False
        self.value = value
        self.steps = 0
        if cost is None:
            self.cost = {'inspect':1, 'control':1, 'boot':1, 'discover': 1}
        else:
            self.cost = cost

    def turn(self):
        """Implements the player's strategy.

        Parameters
        ----------
        None

        Raises
        ------
        TBD
        """
        self.steps += 1
        
        if self.goal_achieved:
            return (None, "stop")
        
        if self.steps > 3:
            return (None, "stop")
        
        # get nodes in play
        ctrl = nx.get_node_attributes(self.topology, "attacker_control")
        ctrl = [k for k,v in ctrl.items() if v]
        peers = set()
        for node in ctrl:
            peers = peers.union([n for n in self.topology.neighbors(node)])
        # choose a random target to take over
        potential_targets = list(peers.difference(ctrl))
        if len(potential_targets) <= 0:
            return (None, "stop")
        else:
            target = random.choice(potential_targets)
            self.topology.nodes[target]["attacker_control"] = True
            self.value -= self.cost['control']
            return (target, "control")

    def inspect(self, node, properties):
        """Updates the player's understanding of the topology with the additional node properties.

        Parameters
        ----------
        node : str
            The inspected node name
        properties: dict
            The properties of the inspected node (note: the game may limit shared properties)

        Raises
        ------
        TBD
        """
        self.topology.nodes[node]['attacker_control'] = properties.get('attacker_control', None)
        self.topology.nodes[node]['defender_control'] = properties.get('defender_control', None)
        
    def discover(self, node, edges):
        """Updates the player's understanding of the topology with the additional edges.

        Parameters
        ----------
        node : str
            The discovered node name
        edges: list
            The list of tuples of edges associated with the discovered node

        Raises
        ------
        TBD
        """
        self.topology.add_edges_from(edges)
        
    def goal(self, node, value):
        """Updates the player's understanding of the topology with the attacker control of the associated node.

        Parameters
        ----------
        node : str
            Identifies that the attacker has controlled the associated goal node.
        value: float
            The value of the goal

        Raises
        ------
        TBD
        """
        self.goal_achieved = True

### BFS Attacker  
Search the first found nodes first

In [591]:
class attacker_bfs_c:
    """
    A class used to represent a player
    
    This class implements a breath first seach for the goal

    ...

    Attributes
    ----------
    topology : nx.classes.graph.Graph
        The network topology on which the game is played as observed by the player
    value : int
        Player's internal representation of the cost of the game
    cost : dict
        Dictionary like {'inspect':1, 'control':1, 'boot':1, 'discover': 1} representing player's understanding of costs of each action
    goal_achieved : bool
        Has the game reported a goal being acheived by the attacker.

    Methods
    -------
    turn()
        chooses an action and returns tuple (node, action)
    inspect(node, properties):
        updates the internal understanding of the topology with the inspected properties on the associated node.
    discover(node, edges):
        updates the internal understanding of the topology with the discovered edges for the associated node.
    goal(node):
        updates the goal as achieved
    """
    
    topology = None
    goal_achieved = None
    value = None
    cost = None
    to_explore = None

    def __init__(self, topology, value=0, cost=None):
        """
        Parameters
        ----------
        topology : nx.classes.graph.Graph
            The network topology on which the game is played as observed by the player
        value : int
            Player's initial starting value
        cost : dict
            Dictionary like {'inspect':1, 'control':1, 'boot':1, 'discover': 1} representing player's understanding of costs of each action
        """
        #raise NotImplementedError("DFS attacker not yet implemented")
        
        self.topology = topology
        self.goal_achieved = False
        self.value = value
        self.to_explore = list()
        if cost is None:
            self.cost = {'inspect':1, 'control':1, 'boot':1, 'discover': 1}
        else:
            self.cost = cost

    def turn(self):
        """Implements the player's strategy.

        Parameters
        ----------
        None

        Raises
        ------
        TBD
        """
        if self.goal_achieved:
            return (None, "stop")
        
        # get nodes in play
        ctrl = nx.get_node_attributes(self.topology, "attacker_control")
        ctrl = [k for k,v in ctrl.items() if v]
        for node in ctrl:
            peers = list(self.topology.neighbors(node))
            random.shuffle(peers)
            for peer in peers:
                if peer not in ctrl and peer not in self.to_explore:
                    self.to_explore.append(peer)
        # choose a random target to take over
        if len(self.to_explore) <= 0:
            return (None, "stop")
        target = self.to_explore.pop(0)
        self.topology.nodes[target]["attacker_control"] = True
        self.value -= self.cost['control']
        return (target, "control")

    def inspect(self, node, properties):
        """Updates the player's understanding of the topology with the additional node properties.

        Parameters
        ----------
        node : str
            The inspected node name
        properties: dict
            The properties of the inspected node (note: the game may limit shared properties)

        Raises
        ------
        TBD
        """
        self.topology.nodes[node]['attacker_control'] = properties.get('attacker_control', None)
        self.topology.nodes[node]['defender_control'] = properties.get('defender_control', None)
        
    def discover(self, node, edges):
        """Updates the player's understanding of the topology with the additional edges.

        Parameters
        ----------
        node : str
            The discovered node name
        edges: list
            The list of tuples of edges associated with the discovered node

        Raises
        ------
        TBD
        """
        self.topology.add_edges_from(edges)
        
    def goal(self, node, value):
        """Updates the player's understanding of the topology with the attacker control of the associated node.

        Parameters
        ----------
        node : str
            Identifies that the attacker has controlled the associated goal node.
        value: float
            The value of the goal

        Raises
        ------
        TBD
        """
        self.goal_achieved = True

## Defenders

### Basic Defender  
Randomly inspects a node and if it is controlled by the attacker, boots the attacker.

In [592]:
### Define look and mitigate Defender
class defender_c:
    """
    A class used to represent a player
    
    This class implements a basic defender who randomly chooses a node, inspects it, and if
    controlled by the attacker, boots the attacker.

    ...

    Attributes
    ----------
    topology : nx.classes.graph.Graph
        The network topology on which the game is played as observed by the player
    value : int
        Player's internal representation of the cost of the game
    cost : dict
        Dictionary like {'inspect':1, 'control':1, 'boot':1, 'discover': 1} representing player's understanding of costs of each action
    goal_achieved : bool
        Has the game reported a goal being acheived by the attacker.

    Methods
    -------
    turn()
        chooses an action and returns tuple (node, action)
    inspect(node, properties):
        updates the internal understanding of the topology with the inspected properties on the associated node.
    discover(node, edges):
        updates the internal understanding of the topology with the discovered edges for the associated node.
    goal(node):
        updates the goal as achieved
    """
    
    topology = None
    node_to_mitigate = None
    goal_achieved = None
    value = None
    boot = None

    def __init__(self, topology, value=10, cost = None):
        """
        Parameters
        ----------
        topology : nx.classes.graph.Graph
            The network topology on which the game is played as observed by the player
        value : int
            Player's initial starting value
        cost : dict
            Dictionary like {'inspect':1, 'control':1, 'boot':1, 'discover': 1} representing player's understanding of costs of each action
        """
        self.topology = topology
        self.goal_achieved = False
        self.value = value # we need to pick when to stop so we'll set it to 10 because that's the 
        if cost is None:
            self.cost = {'inspect': 1, 'control': 1, 'boot': 1, 'discover':1}
        else:
            self.cost = cost

    def turn(self):
        """Implements the player's strategy.

        Parameters
        ----------
        None

        Raises
        ------
        TBD
        """
        if self.goal_achieved:
            return (None, "stop")
        elif self.value <= 0:
            return (None, "stop")
        # if we don't have a target, inspect one
        elif self.node_to_mitigate is None:
            ctrl = nx.get_node_attributes(self.topology, "defender_control")
            ctrl = [k for k,v in ctrl.items() if v]
            peers = set(ctrl)
            for node in ctrl:
                peers = peers.union([n for n in self.topology.neighbors(node)])
            # choose a random target to inspect
            target = random.choice(list(peers))
            self.value -= self.cost['inspect']
            return (target, "inspect")
        # if we don't control the target, see if we can
        elif not self.topology.nodes[self.node_to_mitigate]['defender_control']:
            ctrl = nx.get_node_attributes(self.topology, "defender_control")
            ctrl = [k for k,v in ctrl.items() if v]
            peers = set(ctrl)
            for node in ctrl:
                peers = peers.union([n for n in self.topology.neighbors(node)])
            # if we can't mitigate it because we don't control a peer, randomly choose something else to inspect
            if self.node_to_mitigate not in peers:
                target = random.choice(list(peers))
                self.value -= self.cost['inspect']
                return (target, "inspect")
            # else take control of it
            else:
                self.value -= self.cost['control']
                return (self.node_to_mitigate, "control")
        # if we control it, boot the attacker
        else:
            ret = (self.node_to_mitigate, "boot")
            self.node_to_mitigate = None
            self.value -= self.cost['boot']
            return ret


    def inspect(self, node, properties):
        """Updates the player's understanding of the topology with the additional node properties.

        Parameters
        ----------
        node : str
            The inspected node name
        properties: dict
            The properties of the inspected node (note: the game may limit shared properties)

        Raises
        ------
        TBD
        """
        self.topology.nodes[node]['attacker_control'] = properties.get('attacker_control', None)
        self.topology.nodes[node]['defender_control'] = properties.get('defender_control', None)
        # if the inspected node is controlled by the attacker, plan ot 
        if properties.get('attacker_control', None):
            self.node_to_mitigate = node
        
    def discover(self, node, edges):
        """Updates the player's understanding of the topology with the additional edges.

        Parameters
        ----------
        node : str
            The discovered node name
        edges: list
            The list of tuples of edges associated with the discovered node

        Raises
        ------
        TBD
        """
        self.topology.add_edges_from(edges)
        
    def goal(self, node, value):
        """Updates the player's understanding of the topology with the attacker control of the associated node.

        Parameters
        ----------
        node : str
            Identifies that the attacker has controlled the associated goal node.
        value: float
            The value of the goal

        Raises
        ------
        TBD
        """
        self.goal_achieved = True

### Do Nothing Defender  
Defender takes no action.  Saves cost.

In [593]:
### Define do-thing Defender
class defender_do_nothing_c:
    """
    A class used to represent a player
    
    This class implements a defender who takes no actions.

    ...

    Attributes
    ----------
    topology : nx.classes.graph.Graph
        The network topology on which the game is played as observed by the player
   goal_achieved : bool
        Has the game reported a goal being acheived by the attacker.

    Methods
    -------
    turn()
        chooses an action and returns tuple (node, action)
    inspect(node, properties):
        updates the internal understanding of the topology with the inspected properties on the associated node.
    discover(node, edges):
        updates the internal understanding of the topology with the discovered edges for the associated node.
    goal(node):
        updates the goal as achieved
    """
    
    topology = None
    goal_achieved = None

    def __init__(self, topology, value=None, cost=None):
        """
        Parameters
        ----------
        topology : nx.classes.graph.Graph
            The network topology on which the game is played as observed by the player
        value : int
            Player's initial starting value
        cost : dict
            Dictionary like {'inspect':1, 'control':1, 'boot':1, 'discover': 1} representing player's understanding of costs of each action
        """
        self.topology = topology
        self.goal_achieved = False

    def turn(self):
        """Implements the player's strategy.

        Parameters
        ----------
        None

        Raises
        ------
        TBD
        """
        return (None, "stop")


    def inspect(self, node, properties):
        """Updates the player's understanding of the topology with the additional node properties.

        Parameters
        ----------
        node : str
            The inspected node name
        properties: dict
            The properties of the inspected node (note: the game may limit shared properties)

        Raises
        ------
        TBD
        """
        self.topology.nodes[node]['attacker_control'] = properties.get('attacker_control', None)
        self.topology.nodes[node]['defender_control'] = properties.get('defender_control', None)
        
    def discover(self, node, edges):
        """Updates the player's understanding of the topology with the additional edges.

        Parameters
        ----------
        node : str
            The discovered node name
        edges: list
            The list of tuples of edges associated with the discovered node

        Raises
        ------
        TBD
        """
        self.topology.add_edges_from(edges)
        
    def goal(self, node, value):
        """Updates the player's understanding of the topology with the attacker control of the associated node.

        Parameters
        ----------
        node : str
            Identifies that the attacker has controlled the associated goal node.
        value: float
            The value of the goal

        Raises
        ------
        TBD
        """
        self.goal_achieved = True


### Monitor and Hunt Defender  
Monitors the internet connected systems (checks if attacker controlled).  If so, it mitigates it and starts checking the neighbor nodes to see if they are attacker controlled.

In [594]:
### Define monitor and hunt
class defender_MandH_c:
    """
    A class used to represent a player
    
    This class implements a defender who monitors the internet connected systems and hunts and remediates from there.

    ...

    Attributes
    ----------
    topology : nx.classes.graph.Graph
        The network topology on which the game is played as observed by the player
    value : int
        Player's internal representation of the cost of the game
    cost : dict
        Dictionary like {'inspect':1, 'control':1, 'boot':1, 'discover': 1} representing player's understanding of costs of each action
    goal_achieved : bool
        Has the game reported a goal being acheived by the attacker.

    Methods
    -------
    turn()
        chooses an action and returns tuple (node, action)
    inspect(node, properties):
        updates the internal understanding of the topology with the inspected properties on the associated node.
    discover(node, edges):
        updates the internal understanding of the topology with the discovered edges for the associated node.
    goal(node):
        updates the goal as achieved
    """
    
    topology = None
    node_to_mitigate = None
    goal_achieved = None
    value = None
    boot = None
    boundary = None
    hunt = None

    def __init__(self, topology, value=10, cost = None):
        """
        Parameters
        ----------
        topology : nx.classes.graph.Graph
            The network topology on which the game is played as observed by the player
        value : int
            Player's initial starting value
        cost : dict
            Dictionary like {'inspect':1, 'control':1, 'boot':1, 'discover': 1} representing player's understanding of costs of each action
        """
        self.topology = topology
        self.goal_achieved = False
        self.value = value # we need to pick when to stop so we'll set it to 10 because that's the 
        if cost is None:
            self.cost = {'inspect': 1, 'control': 1, 'boot': 1, 'discover':1}
        else:
            self.cost = cost
        self.boundary = {"webapp1": None, "webapp2": None, "mail": None, "proxy": None} # True if atkr controlled
        self.hunt = set()
        
        
    def turn(self):
        """Implements the player's strategy.

        Parameters
        ----------
        None

        Raises
        ------
        TBD
        """
        if self.goal_achieved:
            return (None, "stop")
        elif self.value <= 0:
            return (None, "stop")
 
        # if we don't have a target, inspect one
        elif self.node_to_mitigate is None:
            potential_nodes = list(self.hunt.union([k for k,v in self.boundary.items() if v is not False]))
            if len(potential_nodes) <= 0:
                return (None, "stop")
            else:
                target = random.choice(potential_nodes)
                self.value -= self.cost['inspect']
                return (target, "inspect")
        # if we don't control the target, see if we can
        elif not self.topology.nodes[self.node_to_mitigate]['defender_control']:
            ctrl = nx.get_node_attributes(self.topology, "defender_control")
            ctrl = [k for k,v in ctrl.items() if v]
            peers = set(ctrl)
            for node in ctrl:
                peers = peers.union([n for n in self.topology.neighbors(node)])
            # if we can't mitigate it because we don't control a peer, randomly choose something else to inspect
            if self.node_to_mitigate not in peers:
                potential_nodes = list(self.hunt.union([k for k,v in self.boundary.items() if v is not False]))
                if len(potential_nodes) <= 0:
                    return (None, "stop")
                else:
                    target = random.choice(potential_nodes)
                    self.value -= self.cost['inspect']
                    return (target, "inspect")
            # else take control of it
            else:
                self.value -= self.cost['control']
                return (self.node_to_mitigate, "control")
        # if we control it, boot the attacker
        else:
            ret = (self.node_to_mitigate, "boot")
            self.hunt = self.hunt.difference(self.node_to_mitigate)
            self.node_to_mitigate = None
            self.value -= self.cost['boot']
            return ret


    def inspect(self, node, properties):
        """Updates the player's understanding of the topology with the additional node properties.

        Parameters
        ----------
        node : str
            The inspected node name
        properties: dict
            The properties of the inspected node (note: the game may limit shared properties)

        Raises
        ------
        TBD
        """
        self.topology.nodes[node]['attacker_control'] = properties.get('attacker_control', None)
        self.topology.nodes[node]['defender_control'] = properties.get('defender_control', None)
        # if the inspected node is controlled by the attacker, plan ot 
        if properties.get('attacker_control', None) is True:
            self.node_to_mitigate = node
#            neighbors = set(self.topology.neighbors(node))
#            ctrl = nx.get_node_attributes(self.topology, "defender_control")
#            self.hunt = self.hunt.union(neighbors.difference(ctrl))
            self.hunt = self.hunt.union(set(self.topology.neighbors(node)).difference(["internet"]))
        else:
            self.hunt = self.hunt.difference([node])
        if node in self.boundary:
            self.boundary[node] = properties.get('attacker_control', None)
        
        
    def discover(self, node, edges):
        """Updates the player's understanding of the topology with the additional edges.

        Parameters
        ----------
        node : str
            The discovered node name
        edges: list
            The list of tuples of edges associated with the discovered node

        Raises
        ------
        TBD
        """
        self.topology.add_edges_from(edges)
        
        
    def goal(self, node, value):
        """Updates the player's understanding of the topology with the attacker control of the associated node.

        Parameters
        ----------
        node : str
            Identifies that the attacker has controlled the associated goal node.
        value: float
            The value of the goal

        Raises
        ------
        TBD
        """
        self.goal_achieved = True

# GAMES AND ANALYSIS

## One Game

In [541]:
game = complex_competition(attacker_least_knowledge_c, defender_do_nothing_c)
game.play()
game.value

{'attacker': 1, 'defender': -10}

## attacker cheats and defender does nothing

In [265]:
data = []
for i in range(10000):
    game = complex_competition(attacker_shortest_path_c, defender_do_nothing_c)
    #data.append([game.value['attacker'], game.value['defender']])
    game.play()
    data.append([game.value['attacker'], game.value['defender']])
    #del(game)

In [268]:
#alt.Chart(pd.DataFrame(data, columns = ['attacker value', 'defender value'])).mark_circle(size=60).encode(
#    x='attacker value',
#    y='defender value',
#    tooltip=['attacker value', 'defender value']
#).interactive()

alt.Chart(pd.DataFrame(data, columns = ['attacker value', 'defender value']).sample(n=5000)).mark_bar().encode(
    alt.X('attacker value', bin=True),
    y='count()',
)

alt.Chart(...)

In [267]:
pd.DataFrame(data, columns = ['attacker value', 'defender value']).value_counts()

attacker value  defender value
5               -10               10000
dtype: int64

## Attacker random Defender do nothing

In [ ]:
alt.Chart(pd.DataFrame(data, columns = ['attacker value', 'defender value']).sample(n=5000)).mark_bar().encode(
    alt.X('attacker value', bin=True),
    y='count()',
)

In [ ]:
data = []
for i in range(10000):
    game = complex_competition(attacker_c, defender_do_nothing_c)
    #data.append([game.value['attacker'], game.value['defender']])
    game.play()
    data.append([game.value['attacker'], game.value['defender']])
    #del(game)

In [ ]:
pd.DataFrame(data, columns = ['attacker value', 'defender value']).value_counts()

## shortest_path vs attacker value

In [ ]:
### Look for a relationship between path length and cost to attack

data = []
for i in range(5000):
    game = complex_competition(attacker_c, defender_do_nothing_c)
    g = game.topology
    nx.set_node_attributes(game.topology, False, "goal")
    nodes = set(game.topology.nodes()).difference(["internet"])
    goal = random.choice(list(nodes))
    game.topology.nodes[goal]['goal'] = True
    shortest_path = nx.shortest_path_length(game.topology, "internet", goal)
    game.play()
    data.append([shortest_path, game.value['attacker']])
    #del(game)

In [317]:
alt.Chart(pd.DataFrame(data, columns = ['shortest path', 'attacker value'])).mark_circle(size=60).encode(
    x='shortest path',
    y='attacker value',
    tooltip=['shortest path', 'attacker value']
).interactive()

alt.Chart(...)

## shortest_path vs attacker value (w/o backtracing)

In [318]:
### Look for a relationship between path length and cost to attack

data = []
for i in range(5000):
    game = complex_competition(attacker_no_backtrack_c, defender_do_nothing_c)
    g = game.topology
    nx.set_node_attributes(game.topology, False, "goal")
    goal = random.choice(list(game.topology.nodes()))
    game.topology.nodes[goal]['goal'] = True
    shortest_path = nx.shortest_path_length(game.topology, "internet", goal)
    game.play()
    data.append([shortest_path, game.value['attacker']])
    #del(game)

In [319]:
alt.Chart(pd.DataFrame(data, columns = ['shortest path', 'attacker value'])).mark_circle(size=60).encode(
    x='shortest path',
    y='attacker value',
    tooltip=['shortest path', 'attacker value']
).interactive()

alt.Chart(...)

## Attacker w/o Backtrace vs Monitor and Hunt - relative values  

In [463]:
game = complex_competition(attacker_no_backtrack_c, defender_MandH_c, d_params={"value": 10000}, a_params={"value": 10})
#g = game.topology
#nx.set_node_attributes(game.topology, False, "goal")
#nodes = set(game.topology.nodes()).difference(["internet"])
#goal = random.choice(list(nodes))
#game.topology.nodes[goal]['goal'] = True
#shortest_path = nx.shortest_path_length(game.topology, "internet", goal)
game.play()

In [476]:
### Look for a relationship between path length and cost to attack

data = []
for i in range(1000):
    game = complex_competition(attacker_no_backtrack_c, defender_MandH_c, d_params={"value": 10000}, a_params={"value": 30})
    #g = game.topology
    #nx.set_node_attributes(game.topology, False, "goal")
    #nodes = set(game.topology.nodes()).difference(["internet"])
    #goal = random.choice(list(nodes))
    #game.topology.nodes[goal]['goal'] = True
    #shortest_path = nx.shortest_path_length(game.topology, "internet", goal)
    game.play()
    data.append([game.value['defender'], game.value['attacker'], game.attacker.goal_achieved])
    #del(game)

Actor attacker attempted to act on node infra1 but it is not a peer so action had no effect.
Actor attacker attempted to act on node svr1_3 but it is not a peer so action had no effect.
Actor attacker attempted to act on node svr2_3 but it is not a peer so action had no effect.
Actor attacker attempted to act on node infra1 but it is not a peer so action had no effect.
Actor attacker attempted to act on node svr1_2 but it is not a peer so action had no effect.
Actor attacker attempted to act on node svr2_3 but it is not a peer so action had no effect.
Actor attacker attempted to act on node mgmt1 but it is not a peer so action had no effect.
Actor attacker attempted to act on node pci1 but it is not a peer so action had no effect.
Actor attacker attempted to act on node pci3 but it is not a peer so action had no effect.
Actor attacker attempted to act on node pci2 but it is not a peer so action had no effect.
Actor attacker attempted to act on node pci1 but it is not a peer so action h

In [477]:
alt.Chart(pd.DataFrame(data, columns = ['defender value', 'attacker value', 'goal']))\
.mark_circle(size=60, opacity=0.2)\
.encode(
    x='defender value',
    y='attacker value',
    color='goal',
    tooltip=['defender value', 'attacker value']
).interactive()


alt.Chart(...)

In [478]:
cnt = {"attack": 0, "defense": 0}
for row in data:
    if row[2]:
        cnt['attack'] += 1
    else:
        cnt['defense'] += 1
        
print(cnt)

{'attack': 880, 'defense': 120}


## topo vs strategy vs value ratio

What is the relationship between topology, attacker strategy, attacker action cost to goal value ratio?  

* Topologies:
  * Goal in servers
  * Goal is DB (in DMZ)
  * Goal is client
  * Goal is pci
* Strategies:
  * Short attacks
  * Least-knowledge
  * DFS
  * Random no backtrack
  * BFS
* Ratio:
  * 1:5 # min with 5 node shortet path
  * 1:10
  * 1:26 # max w/ 25 nodes
  
  4 * 4 * 3 = 36 combinations
  5000 iterations / 36 combinations = 138 iterations

In [560]:
g = nx.Graph()

## outside node
outside = ["internet"]
# DMZ: 2 webapp and 2 database
dmz = ['webapp1', 'db1', "webapp2", "db2", "mail", "proxy"]
# Userlan: 5 users
users = ["user1", "user2", "user3", "user4", "user5"]
# 3 mgmt systems
mgmt = ["mgmt1", "mgmt2", "mgmt3"]
# 3 infrastructure systems
infra = ["infra1", "infra2", "infra3"]
# Servers (7 servers in three permission groups)
svrs1 = ["svr1_1", "svr1_2", "svr1_3"] 
svrs2 = ["svr2_1", "svr2_2", "svr2_3"]
svrs3 = ["svr3_1"]
# PCI: 3 PCI related servers
pci = ["pci1", "pci2", "pci3"]
# add nodes
g.add_nodes_from(dmz + users + mgmt + infra + svrs1 + svrs2 + svrs3 + pci)

## add edges
# internet access to dmz webapps
g.add_edges_from([("internet", "webapp1"), ("internet", "webapp2"), 
                  ("internet", "mail"), ("internet", "proxy")])
# internet access to all clients
nx.add_star(g, ["mail"] + users)
nx.add_star(g, ["proxy"] + users)
# Webapps are connected to their database
g.add_edges_from([('webapp1', 'db1'), ("webapp2", "db2")])
# all clients are in the same security domain
edges = itertools.combinations(users, 2)
g.add_edges_from(edges)
# add server groups
edges = itertools.combinations(svrs1, 2)
g.add_edges_from(edges)
edges = itertools.combinations(svrs2, 2)
g.add_edges_from(edges)
# Connect users to svr 1 in each group
nx.add_star(g, svrs1[:1] + users)
nx.add_star(g, svrs2[:1] + users)
nx.add_star(g, svrs3[:1] + users)
# Connect users to webapps
nx.add_star(g, ["webapp1"] + users)
nx.add_star(g, ["webapp2"] + users)
# mgmt is only accessable from 1 user but accesses every server
nx.add_star(g, users[-1:] + mgmt)
for node in mgmt:
    nx.add_star(g, [node] + dmz + svrs1 + svrs2 + svrs3 + infra)
# infra is only accessable from mgmt, but connects to all other servers
for node in infra:
    nx.add_star(g, [node] + dmz + svrs1 + svrs2 + svrs3)
# PCI connects to 1 server and then to itself
g.add_edge(svrs3[0], pci[0])
edges = itertools.combinations(pci, 2)
g.add_edges_from(edges)
g.add_edge("webapp2", svrs3[0]) # webapp connected to PCI

## Set initial control: controller controls everything but internet, attacker also controls the internet
attr = {node:(True if node == "internet" else False) for node in g.nodes()}
nx.set_node_attributes(g, attr, 'attacker_control')
attr = {node:(True if node != "internet" else False) for node in g.nodes()}
nx.set_node_attributes(g, attr, 'defender_control')

In [ ]:
from pert import PERT

In [581]:
data = []
for i in range(150):
    for topo in ["svr", "db", "client", "pci"]:
        for strat in ["short", "know", "dfs", "rando", "bfs"]:
            for ratio in [5, 10, 26]:
                # assign a topo (goal)
                topology = copy.deepcopy(g)
                if topo == "svr":
                    attr = {node:(True if node == "svr1_1" else False) for node in topology.nodes()}
                    nx.set_node_attributes(topology, attr, 'goal')
                elif topo == "db":
                    attr = {node:(True if node == "db1" else False) for node in topology.nodes()}
                    nx.set_node_attributes(topology, attr, 'goal')
                elif topo == "client":
                    attr = {node:(True if node == "user1" else False) for node in topology.nodes()}
                    nx.set_node_attributes(topology, attr, 'goal')
                else: # topo == "pci":
                    attr = {node:(True if node == "pci3" else False) for node in topology.nodes()}
                    nx.set_node_attributes(topology, attr, 'goal')
                    
                # Assign a strategy
                if strat == "short":
                    atkr = attacker_three_and_out_c
                elif strat == "know":
                    atkr = attacker_least_knowledge_c
                elif strat == "dfs":
                    atkr = attacker_dfs_c
                elif strat == "bfs":
                    atkr = attacker_bfs_c
                else:
                    atkr = attacker_no_backtrack_c
                
                # asign goal ratio by setting goal value
                if ratio == 5:
                    goal = 5
                elif ratio == 10:
                    goal = 10
                elif ratio == 26:
                    goal = 26
                    
                # run game and record data
                game = complex_competition(a=atkr, d=defender_do_nothing_c, topology=topology, goal_value=goal)
                game.play()
                data.append([topo, strat, ratio, game.value['attacker'], game.value['defender']])


In [582]:
pd.DataFrame(data, columns = ['topology', 'strategy', 'ratio', 'goal', 'attacker value', 'defender value']).to_parquet(path="./attacker_value.parquet", engine="fastparquet", compression="gzip")

In [563]:
### comparison of pay-outs by factors
#attacker_value %>%
#    mutate(`defender value` = as.factor(`defender value`)) %>%
#    glimpse() %>%
#  ggplot(aes(y=0, x=`attacker value`, color=`defender value`)) +
#    ggbeeswarm::geom_quasirandom(groupOnX=FALSE) +
#    facet_grid(topology ~ strategy) +
#    ggthemes::scale_color_tableau()

-3

In [566]:
### Strategy value over time
#attacker_value %>%
#    glimpse() %>%
#    group_by(strategy) %>%
#      mutate(running_sum = cumsum(`attacker value`)) %>%
#      mutate(run = 1:n()) %>%
#    ungroup() %>%
#  ggplot() +
#    geom_line(aes(run, running_sum, group=strategy, color=strategy)) +
#    ggthemes::scale_color_tableau() +
#    theme(legend.position="bottom")

[['svr', 'short', 5, -3, 0],
 ['svr', 'short', 10, -3, 0],
 ['svr', 'short', 26, -3, 0],
 ['svr', 'know', 5, -7, -5],
 ['svr', 'know', 10, -10, -10],
 ['svr', 'know', 26, 7, -26],
 ['svr', 'dfs', 5, -5, -5],
 ['svr', 'dfs', 10, 7, -10],
 ['svr', 'dfs', 26, 14, -26],
 ['svr', 'rando', 5, -4, -5]]

## topo vs strategy vs value ratio

What is the relationship between topology, attacker strategy, attacker action cost to goal value ratio?  

* Topologies:

  * Goal in servers
  * Goal is DB (in DMZ)
  * Goal is client
  * Goal is pci
* Strategies:
  * Short attacks
  * Least-knowledge
  * DFS
  * Random no backtrack
  * BFS
* Ratio:
  * PERT 0, 4, 54

In [596]:
g = nx.Graph()

## outside node
outside = ["internet"]
# DMZ: 2 webapp and 2 database
dmz = ['webapp1', 'db1', "webapp2", "db2", "mail", "proxy"]
# Userlan: 5 users
users = ["user1", "user2", "user3", "user4", "user5"]
# 3 mgmt systems
mgmt = ["mgmt1", "mgmt2", "mgmt3"]
# 3 infrastructure systems
infra = ["infra1", "infra2", "infra3"]
# Servers (7 servers in three permission groups)
svrs1 = ["svr1_1", "svr1_2", "svr1_3"] 
svrs2 = ["svr2_1", "svr2_2", "svr2_3"]
svrs3 = ["svr3_1"]
# PCI: 3 PCI related servers
pci = ["pci1", "pci2", "pci3"]
# add nodes
g.add_nodes_from(dmz + users + mgmt + infra + svrs1 + svrs2 + svrs3 + pci)

## add edges
# internet access to dmz webapps
g.add_edges_from([("internet", "webapp1"), ("internet", "webapp2"), 
                  ("internet", "mail"), ("internet", "proxy")])
# internet access to all clients
nx.add_star(g, ["mail"] + users)
nx.add_star(g, ["proxy"] + users)
# Webapps are connected to their database
g.add_edges_from([('webapp1', 'db1'), ("webapp2", "db2")])
# all clients are in the same security domain
edges = itertools.combinations(users, 2)
g.add_edges_from(edges)
# add server groups
edges = itertools.combinations(svrs1, 2)
g.add_edges_from(edges)
edges = itertools.combinations(svrs2, 2)
g.add_edges_from(edges)
# Connect users to svr 1 in each group
nx.add_star(g, svrs1[:1] + users)
nx.add_star(g, svrs2[:1] + users)
nx.add_star(g, svrs3[:1] + users)
# Connect users to webapps
nx.add_star(g, ["webapp1"] + users)
nx.add_star(g, ["webapp2"] + users)
# mgmt is only accessable from 1 user but accesses every server
nx.add_star(g, users[-1:] + mgmt)
for node in mgmt:
    nx.add_star(g, [node] + dmz + svrs1 + svrs2 + svrs3 + infra)
# infra is only accessable from mgmt, but connects to all other servers
for node in infra:
    nx.add_star(g, [node] + dmz + svrs1 + svrs2 + svrs3)
# PCI connects to 1 server and then to itself
g.add_edge(svrs3[0], pci[0])
edges = itertools.combinations(pci, 2)
g.add_edges_from(edges)
g.add_edge("webapp2", svrs3[0]) # webapp connected to PCI

## Set initial control: controller controls everything but internet, attacker also controls the internet
attr = {node:(True if node == "internet" else False) for node in g.nodes()}
nx.set_node_attributes(g, attr, 'attacker_control')
attr = {node:(True if node != "internet" else False) for node in g.nodes()}
nx.set_node_attributes(g, attr, 'defender_control')

In [602]:
# set up pert distribution from 0 to 52 with a mode of 4
# 0 is no value
# 54 is double the max effort in the network (explore all nodes)
# 4 is the diameter of the network (minimum effort for max protected asset)
diameter = nx.algorithms.distance_measures.diameter(g)
n_nodes = len(list(g.nodes())) - 1 # minus 1 for attacker starting point
# set up relatively arbitrary (sme best guess) pay-outs per node type
client_pert = PERT(0,1,4)
svr_pert = PERT(0, diameter, n_nodes)
db_pert = PERT(0, diameter, n_nodes)
pci_pert = PERT(10, n_nodes, n_nodes * 2)
# set up relatively aritrary (sme best guess) probability of goal existing
goals = {
    "client": 0.9,
    "server": 0.5,
    "db": 0.75,
    "pci": 0.1
}

data = []
for i in range(1000):
    for strat in ["short", "know", "dfs", "rando", "bfs"]:
        # assign a topo (goal)
        topology = copy.deepcopy(g)
        attr = {node:0 for node in topology.nodes()}
        if random.random() < goals['server']:
            attr["svr1_1"] = float(svr_pert.rvs(1))
        if random.random() < goals['db']:
            attr['db1'] = float(db_pert.rvs(1))
        if random.random() < goals['client']:
            attr['user1'] = float(client_pert.rvs(1))
        if random.random() < goals['pci']:
            attr['pci3'] = float(pci_pert.rvs(1))
        nx.set_node_attributes(topology, attr, 'goal')

        # Assign a strategy
        if strat == "short":
            atkr = attacker_three_and_out_c
        elif strat == "know":
            atkr = attacker_least_knowledge_c
        elif strat == "dfs":
            atkr = attacker_dfs_c
        elif strat == "bfs":
            atkr = attacker_bfs_c
        else:
            atkr = attacker_no_backtrack_c

        # asign goal ratio by setting goal value and fixing costs at 1
        #goal = float(pert.rvs(1))

        # run game and record data
        game = complex_competition(a=atkr, d=defender_do_nothing_c, topology=topology, goal_value=goal)
        game.play()

        data.append([
            i, 
            json.dumps({k:v for k,v in attr.items() if v > 0}), 
            json.dumps(game.goals_achieved),
            strat, game.attacker.goal_achieved, game.value['attacker'], game.value['defender']
        ])


In [604]:
pd.DataFrame(data, columns = ['id', 'goals', 'goals_achieved', 'strategy', 'goal_achieved', 'attacker value', 'defender value']).to_parquet(path="./attacker_value2.parquet", engine="fastparquet", compression="gzip")

In [ ]:
#attacker_value2 = arrow::read_parquet("/Users/v685573/Documents/Development/complex_competition/attacker_value2.parquet")

#attacker_value2 %>%
#    glimpse() %>%
#    group_by(strategy) %>%
#      mutate(running_sum = cumsum(`attacker value`)) %>%
#      mutate(run = 1:n()) %>%
#    ungroup() %>%
#  ggplot() +
#    geom_line(aes(run, running_sum, group=strategy, color=strategy)) +
#    ggthemes::scale_color_tableau() +
#    theme(legend.position="bottom")